# Test

In [19]:
import numpy as np

In [46]:
import numpy as np

# Define the range
low, high = 2, 100

# Dimensions of the 2D array
rows, cols = 3, 4

# Generate the 2D array
random_array = np.random.randint(low, high, size=(rows, cols))

print(random_array)

npMatrix = None

npMatrix = np.random.uniform( size=(rows, cols)) * 100

print(npMatrix)


[[76 78 59 21]
 [19 65 67 10]
 [20 23 95 44]]
[[17.30582427 54.0861309  23.54602144 98.01657404]
 [24.84303278 49.70947551 41.11039614 45.12741736]
 [72.2673843  50.06489548 89.95535533 40.36560467]]


In [ ]:
class Matrix:
    
    def __init__( self, 
                  rows = None, 
                  cols = None, 
                  low = None, 
                  high = None, 
                  wholeNum = None, 
                  path = None,
                  listMatrix = None ):
        
        self.__rows = rows
        self.__cols = cols
        self.__npMatrix = None

        #insert create methods here
        if path == None and listMatrix == None:
            
            if self.__rows == None:
                pass
            else:
                self.__npMatrix = self.__createRandom()

        elif path == None:
            pass
        else:
            pass
        pass

    def __createRandom( self, 
                        rows, 
                        cols,
                        wholeNum,  
                        low = None, 
                        high = None ):
        
        if low == None:
            if wholeNum:
                return np.random.randint(low = 0, high = 100, size=(rows, cols))
            else:
                return np.random.uniform( size=(rows, cols)) * 100
        else:
            if wholeNum:
                return np.random.randint(low, high, size=(rows, cols))
            else:
                return np.random.uniform(low, high, size=(rows, cols))
        

In [ ]:
class StateMachine:
    
    def __init__(self):
        self.__CurrentState = 1  
        self.__run()
              

    # This is the function for state 1, the home screen
    # Home marks the default screen, and is the only screen to allow a exit from the program
    # Home will return the state to move to.
    def __home(self):

        #intro text and options to move into further states
        print("Welcome to the Matrix Operation Manager!\n\n"
              "To use this program type the number of the option and hit enter.\n"
              "Please select from the following options:\n"
              "1.) Create a Matrix\n"
              "2.) List current Matrices available\n"
              "3.) Perform a Matrix Operation\n"
              "4.) Delete a Matrix\n"
              "5.) Exit"
        )

        #input options and validation call
        options = [ 1, 2, 3, 4, 5]
        userInput = None
        userInput = self.__inputValidation(options, userInput)
        
        #local state dictionary to transform user input into the class wide defined states
        stateDict = { 1: 2, 
                      2: 0,
                      3: 0,
                      4: 0,
                      5: 0}

        return stateDict[userInput]
  

    def __createMatrix(self):
        print("Please select from the following options:\n"
              "1.) Create a random matrix\n"
              "2.) Import a matrix\n"
              "3.) Manually create a matrix\n"
              "4.) Return to home"
        )

        options = [ 1, 2, 3, 4]
        userInput = None
        userInput = self.__inputValidation(options, userInput)

        #local state dictionary to transform user input into the class wide defined states
        stateDict = { 1: 3, 
                      2: 4,
                      3: 5,
                      4: 1}

        return stateDict[userInput]

    def __createRandom(self):
        print(1)

    def __inputValidation(self, options, userInput):
        
        try:
            userInput = int(input())
            if userInput not in options:
                raise Exception
        except:
            while userInput not in options:
                print("Please enter the number of one of the options above:\n ")
                try:
                    userInput = int(input())
                except:
                    print("That is not a whole number.\n")
            
        return userInput

    
    __STATES = { 1: __home, 
                 2: __createMatrix
                }
    
    def __run(self):
        while self.__CurrentState != 0:
            self.__CurrentState = self.__STATES[self.__CurrentState](self)
        
        print("Have a nice day!")
            
        


obj = StateMachine()

Welcome to the Matrix Operation Manager!

To use this program type the number of the option and hit enter.
Please select from the following options:
1.) Create a Matrix
2.) List current Matrices available
3.) Perform a Matrix Operation
4.) Delete a Matrix
5.) Exit
Please enter the number of one of the options above:
 
Have a nice day!
